# Luftdaten data : data cleaning, resampling - mini version

# - RETRYING THIS, just to makes sure we've got the hang of it! 
## Code builds a continuous time tabular version of the luftdaen data, such that the same time period is present for each sensor in the data, regardless of whether each sensor has data for all the time slots. 

## Testing :
- using pd.resample
- constructing a time shift using pandas own tools, rather than my own


#### Reference documents

Resampling time series data with Pandas ( Ben Alex Keen ) 
http://benalexkeen.com/resampling-time-series-data-with-pandas/

Pandas reference manual : 

.at - access df values using nay kind of index, for SINGLE VALUES
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iat - only integer index values for getting/setting SINGLE df values
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iat.html

.loc - 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iloc - purely integer indexed access ( getting/setting ) values 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc

#### methods of filling … 

These are some of the common methods you might use for resampling:

Method	Description

bfill	Backward fill

count	Count of values

ffill	Forward fill

first	First valid data value

last	Last valid data value

max	Maximum data value

mean	Mean of values in time range

median	Median of values in time range

min	Minimum data value

nunique	Number of unique values

ohlc	Opening value, highest value, lowest value, closing value

pad	Same as forward fill

std	Standard deviation of values

sum	Sum of values

var	Variance of values

#### time abbreviations 

Alias	Description

B	Business day

D	Calendar day

W	Weekly

M	Month end

Q	Quarter end

A	Year end

BA	Business year end

AS	Year start

H	Hourly frequency

T, min	Minutely frequency

S	Secondly frequency

L, ms	Millisecond frequency

U, us	Microsecond frequency

N, ns	Nanosecond frequency

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# parameters

# start_time = "2018-12-31 21:58:42"
end_time = "2019-01-01 11:58:42"
# generate this please
start_time = "?????"

time_frequency_for_periods__for_basic_data = "5T"
num_of_time_periods___for_basic_data = 24*20 # 24 hrs * 12 x 5 mins in each hour

# when generating time periods 
sampling_frequency = "3T"



# --- data urls 

curr_url = "????"
nordic_midnight_24_hrs_data__url = "/Users/miska/Documents/open_something/luftdaten/luftdaten_code/luftdaten__make_tabular_data__from_db_data/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"
# nordic_midnight_24_hrs_data__url = "/home/miska/documents/opensomething/luftdaten/dustmin_to_csv__various_code/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"



# set the current data source 
curr_url = nordic_midnight_24_hrs_data__url

In [3]:
# try convert the timestamp in the data, to epoch

in_data = pd.read_csv( curr_url )
in_data.shape

(127109, 7)

In [4]:
### prepare the data a bit

In [5]:
# what have we got? 
in_data.dtypes

sensor_id         int64
sensor_namee     object
lat             float64
lon             float64
timestamp        object
p1              float64
p2              float64
dtype: object

In [6]:
type( in_data['timestamp'][0] ) 

str

In [7]:
# set the timestamp as a timestamp
in_data['timestamp'] = pd.to_datetime( in_data['timestamp'] )

In [8]:
in_data

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011,60.002,17.846,2018-12-31 11:57:22,3.43,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


In [9]:
type( in_data['timestamp'][0] )

pandas._libs.tslibs.timestamps.Timestamp

#### slice things up a bit 

In [10]:
in_data__start = in_data[:10]

In [11]:
in_data__start

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011,60.002,17.846,2018-12-31 11:57:22,3.43,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


In [12]:
in_data__middle = in_data[60000:60010]

In [13]:
in_data__middle

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
60000,12679,SDS011,59.384,17.874,2018-12-31 16:52:43,0.40,0.40
60001,12687,SDS011,59.388,17.798,2018-12-31 16:52:21,2.83,1.90
60002,12691,SDS011,57.636,18.304,2018-12-31 16:51:17,2.40,1.70
60003,12693,SDS011,58.190,12.720,2018-12-31 16:51:03,6.42,2.62
60004,14017,SDS011,59.376,18.010,2018-12-31 16:51:10,4.07,1.94
60005,14133,SDS011,59.364,18.018,2018-12-31 16:50:49,4.14,2.38
60006,14209,SDS011,56.070,12.698,2018-12-31 16:51:35,28.52,13.48
60007,14264,SDS011,57.654,11.880,2018-12-31 16:51:51,26.92,8.46
60008,14276,SDS011,57.346,12.150,2018-12-31 16:50:46,72.48,20.16
60009,14278,SDS011,59.374,18.010,2018-12-31 16:51:31,4.64,2.30


In [14]:
in_data__end = in_data[-10:]
in_data__end

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
127099,13020,SDS011,57.722,11.948,2019-01-01 11:59:41,20.64,3.67
127100,16147,SDS011,59.364,18.018,2019-01-01 11:59:46,3.28,1.80
127101,16153,SDS011,55.648,13.208,2019-01-01 11:57:19,20.00,3.90
127102,16296,SDS011,56.144,13.394,2019-01-01 11:59:59,23.86,7.45
127103,16533,SDS011,55.722,13.202,2019-01-01 11:56:55,18.05,4.33
127104,16723,SDS011,57.736,11.894,2019-01-01 11:58:57,16.47,3.40
127105,16815,SDS011,59.462,18.040,2019-01-01 11:59:36,2.67,1.97
127106,17235,SDS011,59.272,17.780,2019-01-01 11:59:41,4.69,1.82
127107,10588,SDS011,55.676,13.346,2019-01-01 11:57:12,14.08,3.68
127108,10647,SDS011,55.608,13.036,2019-01-01 11:58:42,23.42,4.75


#### try putting the different bits together, so you can interpolate some data… 

In [15]:
in_data__beginning_and_middle_and_end = in_data__start.append( in_data__middle ).append( in_data__end )

In [16]:
in_data__beginning_and_middle_and_end

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011,60.002,17.846,2018-12-31 11:57:22,3.43,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


#### now try interpolate the data

In [17]:
# first check that the different datatypes, eg timestamps is in order, and that the timestamps are the index… 

In [18]:
# datatypes
in_data__beginning_and_middle_and_end.dtypes

sensor_id                int64
sensor_namee            object
lat                    float64
lon                    float64
timestamp       datetime64[ns]
p1                     float64
p2                     float64
dtype: object

In [19]:
type( in_data__beginning_and_middle_and_end['timestamp'][0] ) 

pandas._libs.tslibs.timestamps.Timestamp

In [20]:
# ok, that looks ok, 

In [21]:
# but what about the index? 
in_data__beginning_and_middle_and_end.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,  60000,  60001,  60002,  60003,  60004,  60005,
             60006,  60007,  60008,  60009, 127099, 127100, 127101, 127102,
            127103, 127104, 127105, 127106, 127107, 127108],
           dtype='int64')

In [22]:
# alas, this doesn't do the trick, one needs to reference the variable one wants this operation to end up in … 
in_data__beginning_and_middle_and_end.set_index( 'timestamp' )

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:57:22,7273,SDS011,60.002,17.846,3.43,1.56
2018-12-31 11:58:44,7275,SDS011,57.720,11.888,482.77,33.82
2018-12-31 11:58:47,7277,SDS011,59.266,15.230,5.48,2.47
2018-12-31 11:56:41,7406,SDS011,56.964,24.128,11.05,6.62
2018-12-31 11:57:42,7428,SDS011,59.868,17.624,1.78,1.02
2018-12-31 11:57:52,7469,SDS011,56.944,24.142,8.40,5.54
2018-12-31 11:58:51,7597,SDS011,59.320,18.064,3.68,2.00
2018-12-31 11:58:28,8683,SDS011,59.744,18.206,3.01,2.80
2018-12-31 11:57:18,9411,SDS011,59.266,15.230,3.44,2.18


In [23]:
in_data__beginning_and_middle_and_end.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,  60000,  60001,  60002,  60003,  60004,  60005,
             60006,  60007,  60008,  60009, 127099, 127100, 127101, 127102,
            127103, 127104, 127105, 127106, 127107, 127108],
           dtype='int64')

In [24]:
# this should work … 
in_data__beginning_and_middle_and_end = in_data__beginning_and_middle_and_end.set_index( 'timestamp' )


In [25]:
in_data__beginning_and_middle_and_end.index

DatetimeIndex(['2018-12-31 11:57:22', '2018-12-31 11:58:44',
               '2018-12-31 11:58:47', '2018-12-31 11:56:41',
               '2018-12-31 11:57:42', '2018-12-31 11:57:52',
               '2018-12-31 11:58:51', '2018-12-31 11:58:28',
               '2018-12-31 11:57:18', '2018-12-31 11:57:22',
               '2018-12-31 16:52:43', '2018-12-31 16:52:21',
               '2018-12-31 16:51:17', '2018-12-31 16:51:03',
               '2018-12-31 16:51:10', '2018-12-31 16:50:49',
               '2018-12-31 16:51:35', '2018-12-31 16:51:51',
               '2018-12-31 16:50:46', '2018-12-31 16:51:31',
               '2019-01-01 11:59:41', '2019-01-01 11:59:46',
               '2019-01-01 11:57:19', '2019-01-01 11:59:59',
               '2019-01-01 11:56:55', '2019-01-01 11:58:57',
               '2019-01-01 11:59:36', '2019-01-01 11:59:41',
               '2019-01-01 11:57:12', '2019-01-01 11:58:42'],
              dtype='datetime64[ns]', name='timestamp', freq=None)

In [26]:
# now try RESAMPLING! 
in_data__beginning_and_middle_and_end__RESAMPLED = in_data__beginning_and_middle_and_end.resample("5Min").mean().bfill()

In [27]:
in_data__beginning_and_middle_and_end__RESAMPLED

,sensor_id,lat,lon,p1,p2
timestamp,,,,,
2018-12-31 11:55:00,7925.5,58.8428,18.0392,52.516,5.991
2018-12-31 12:00:00,13592.7,58.3782,15.7462,15.282,5.534
2018-12-31 12:05:00,13592.7,58.3782,15.7462,15.282,5.534
2018-12-31 12:10:00,13592.7,58.3782,15.7462,15.282,5.534
2018-12-31 12:15:00,13592.7,58.3782,15.7462,15.282,5.534
2018-12-31 12:20:00,13592.7,58.3782,15.7462,15.282,5.534
2018-12-31 12:25:00,13592.7,58.3782,15.7462,15.282,5.534
2018-12-31 12:30:00,13592.7,58.3782,15.7462,15.282,5.534
2018-12-31 12:35:00,13592.7,58.3782,15.7462,15.282,5.534


#### now let's try making a single data row, of the kind in the data we have

In [28]:
# approach #1 - copy an existing row

In [29]:
single_row = in_data.iloc[0]
single_row

sensor_id                      7273
sensor_namee                 SDS011
lat                          60.002
lon                          17.846
timestamp       2018-12-31 11:57:22
p1                             3.43
p2                             1.56
Name: 0, dtype: object

In [30]:
type( single_row )

pandas.core.series.Series

In [31]:
# testing if one could just add the single row, which turns out to be a series, 
# into the ..... indata.... 
# YES, it looks like it's possible 
in_data.append( single_row )

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011,60.002,17.846,2018-12-31 11:57:22,3.43,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


In [60]:
# let's try modifying the line 
single_row['timestamp']

Timestamp('2018-12-31 11:57:22')

In [61]:
single_row['p1']

0

In [62]:
len( single_row )

7

In [63]:
single_row

sensor_id                      7273
sensor_namee                 SDS011
lat                          60.002
lon                          17.846
timestamp       2018-12-31 11:57:22
p1                                0
p2                             1.56
Name: 0, dtype: object

In [64]:
single_row.index

Index(['sensor_id', 'sensor_namee', 'lat', 'lon', 'timestamp', 'p1', 'p2'], dtype='object')

In [65]:
single_row.p1

0

In [66]:
single_row.p1 * 0

0

In [67]:
single_row['p1'] = np.int( 0 )

/Applications/z_extras/code/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
single_row['p1'].iloc[0] = list( range( len( single_row.index ) ) )

AttributeError: 'int' object has no attribute 'iloc'

In [69]:
in_data_slice = in_data[:10]

In [70]:
in_data_slice

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011_rules,60.002,17.846,2019-02-01 10:54:42,-99.00,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


In [71]:
in_data_slice.iloc[0]['p1']

-99.0

In [72]:
in_data_slice.iloc[:1]['p1'] = in_data_slice.iloc[:1]['p1'] * 0 

/Applications/z_extras/code/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
in_data_slice

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011_rules,60.002,17.846,2019-02-01 10:54:42,-0.00,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


#### let's try using .at
 "Access a single value for a row/column pair by integer position.
Similar to iloc, in that both provide integer-based lookups. Use iat if you only need to get or set a single value in a DataFrame or Series."
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iat.html


In [74]:
in_data_slice.at[ 0, 'p1' ] = -99

In [75]:
in_data_slice

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011_rules,60.002,17.846,2019-02-01 10:54:42,-99.00,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


In [76]:
in_data_slice.columns

Index(['sensor_id', 'sensor_namee', 'lat', 'lon', 'timestamp', 'p1', 'p2'], dtype='object')

In [77]:
# try fetching a value
in_data_slice.at[ 0, "sensor_namee" ]

'SDS011_rules'

In [78]:
# now try setting it 
in_data_slice.at[ 0, "sensor_namee" ] = "SDS011_rules"

In [79]:
in_data_slice

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011_rules,60.002,17.846,2019-02-01 10:54:42,-99.00,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90


In [80]:
# now try setting a timestamp

# - first try making one 
curr_timestamp = pd.Timestamp( time.ctime() ) 
curr_timestamp

Timestamp('2019-02-01 11:01:53')

In [81]:
curr_timestamp = pd.Timestamp( time.ctime() ) 
curr_timestamp

Timestamp('2019-02-01 11:01:55')

In [82]:
type( curr_timestamp )

pandas._libs.tslibs.timestamps.Timestamp

In [83]:
# now let's try setting a timestamp in the data
in_data_slice.at[0, 'timestamp'] = curr_timestamp

In [84]:
in_data_slice

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
0,7273,SDS011_rules,60.002,17.846,2019-02-01 11:01:55,-99.00,1.56
1,7275,SDS011,57.720,11.888,2018-12-31 11:58:44,482.77,33.82
2,7277,SDS011,59.266,15.230,2018-12-31 11:58:47,5.48,2.47
3,7406,SDS011,56.964,24.128,2018-12-31 11:56:41,11.05,6.62
4,7428,SDS011,59.868,17.624,2018-12-31 11:57:42,1.78,1.02
5,7469,SDS011,56.944,24.142,2018-12-31 11:57:52,8.40,5.54
6,7597,SDS011,59.320,18.064,2018-12-31 11:58:51,3.68,2.00
7,8683,SDS011,59.744,18.206,2018-12-31 11:58:28,3.01,2.80
8,9411,SDS011,59.266,15.230,2018-12-31 11:57:18,3.44,2.18
9,9436,SDS011,59.334,18.034,2018-12-31 11:57:22,2.12,1.90



### could this new slice be used for resampling-interpolation? 

#### let's try…

In [85]:
in_data_end02 = in_data[-10:]
in_data_end02

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
127099,13020,SDS011,57.722,11.948,2019-01-01 11:59:41,20.64,3.67
127100,16147,SDS011,59.364,18.018,2019-01-01 11:59:46,3.28,1.80
127101,16153,SDS011,55.648,13.208,2019-01-01 11:57:19,20.00,3.90
127102,16296,SDS011,56.144,13.394,2019-01-01 11:59:59,23.86,7.45
127103,16533,SDS011,55.722,13.202,2019-01-01 11:56:55,18.05,4.33
127104,16723,SDS011,57.736,11.894,2019-01-01 11:58:57,16.47,3.40
127105,16815,SDS011,59.462,18.040,2019-01-01 11:59:36,2.67,1.97
127106,17235,SDS011,59.272,17.780,2019-01-01 11:59:41,4.69,1.82
127107,10588,SDS011,55.676,13.346,2019-01-01 11:57:12,14.08,3.68
127108,10647,SDS011,55.608,13.036,2019-01-01 11:58:42,23.42,4.75


In [86]:
new_later_timestamp = pd.Timestamp( 2019, 1, 1, 15, 0, 0 )
new_later_timestamp

Timestamp('2019-01-01 15:00:00')

In [87]:
new_last_data_row = pd.DataFrame( in_data_end02[-2:] ) 
new_last_data_row

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
127107,10588,SDS011,55.676,13.346,2019-01-01 11:57:12,14.08,3.68
127108,10647,SDS011,55.608,13.036,2019-01-01 11:58:42,23.42,4.75


In [88]:
new_last_data_row.shape

(2, 7)

In [89]:
type( new_last_data_row )

pandas.core.frame.DataFrame

In [91]:
new_last_data_row.shape

(2, 7)

In [104]:
new_last_data_row.columns

Index(['sensor_id', 'sensor_namee', 'lat', 'lon', 'timestamp', 'p1', 'p2'], dtype='object')

In [103]:
new_last_data_row.iloc[0].index

Index(['sensor_id', 'sensor_namee', 'lat', 'lon', 'timestamp', 'p1', 'p2'], dtype='object')

In [122]:
new_last_data_row.iat[ 0, 0 ] = -9999

In [123]:
new_last_data_row

,sensor_id,sensor_namee,lat,lon,timestamp,p1,p2
127107,-9999,SDS011,55.676,13.346,2019-01-01 11:57:12,14.08,3.68
127108,10647,SDS011,55.608,13.036,2019-01-01 11:58:42,23.42,4.75


In [124]:
# NOTE : .at needs to use the index of the DF, now th row count… 
# ok, let's see if we can get .at working again… 
new_last_data_row.at[ 127107, 'p1' ]

14.08

In [126]:
new_last_data_row__SINGLE_ROW_AS_SERIES = new_last_data_row.iloc[0]

In [127]:
new_last_data_row__SINGLE_ROW_AS_SERIES

sensor_id                     -9999
sensor_namee                 SDS011
lat                          55.676
lon                          13.346
timestamp       2019-01-01 11:57:12
p1                            14.08
p2                             3.68
Name: 127107, dtype: object

In [128]:
type( new_last_data_row__SINGLE_ROW_AS_SERIES )

pandas.core.series.Series

In [131]:
new_last_data_row__SINGLE_ROW_AS_SERIES.at['timestamp']

Timestamp('2019-02-01 11:01:55')

In [135]:
# HOWEVER, .at does work on single series, directly!
new_last_data_row__SINGLE_ROW_AS_SERIES.at['timestamp'] = new_later_timestamp

In [ ]:
new_later_timestamp

In [133]:
new_last_data_row__SINGLE_ROW_AS_SERIES

sensor_id                     -9999
sensor_namee                 SDS011
lat                          55.676
lon                          13.346
timestamp       2019-02-01 11:01:55
p1                            14.08
p2                             3.68
Name: 127107, dtype: object

In [139]:
# this could work for setting timestamps, when one wants to use row numbers relative to the index one has
new_last_data_row.iloc[0].at['timestamp'] = new_later_timestamp

In [65]:
new_last_data_row.at[0, 'p1']

KeyError: 0

In [63]:
# in_data_end02__plus_later_timestamp = in_data_end02.append( )

TypeError: append() missing 1 required positional argument: 'other'